In [13]:
import pickle
import pandas as pd
import numpy as np
import tarfile
import os
import pickle
import pandas as pd
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
import gensim



#топ банков по активам (не нашел россельхоз банк)
bank_list = ['sberbank', 'vtb', 'gazprombank', 'vtb24', 'alfabank', 'fk_otkritie', 'mkb']
bank_info_folder = 'TopicModelingFintech'

In [2]:
def unpack_banks(folder_with_archives, bank_list):
    for bank in bank_list:
        path_to_zip_file = folder_with_archives + '/' + bank + '.tar.gz'
        if os.path.isfile(path_to_zip_file):
            if not os.path.isdir(folder_with_archives + '/' + bank):
                tar = tarfile.open(path_to_zip_file, 'r')
                tar.extractall(folder_with_archives)
                tar.close()
        else:
            print(f'{bank} not found.')


In [3]:
unpack_banks(bank_info_folder, bank_list)

In [4]:
# ! pip install deeppavlov

In [5]:
# import deeppavlov as dp
# elmo = dp.build_model(dp.configs.classifiers.rusentiment_elmo, download=True)


In [6]:
branches = pd.read_pickle('TopicModelingFintech/sberbank/branches.pkl')
replies = pd.read_pickle('TopicModelingFintech/sberbank/replies.pkl')
sentences_replies = pd.read_pickle('TopicModelingFintech/sberbank/sentences_replies.pkl')


In [7]:
replies.head(10)

,author,bank,bank_response,comment_page,comments_n,rating,text,time,title,mark
276,ShkolinaElena,sberbank,None,1,0,оценка: 1 проверяется,Снимал деньги в банкомате 20.03.2017 в...,07.04.2017 0:36,Не исправно работающие банкоматы проблема банка,1
280,yanamalkova94,sberbank,None,1,0,оценка: 1 проверяется,06.04.2017г. в 22:55 (дата указана на чеке) мн...,06.04.2017 23:12,"Банкомат забрал деньги и выдал ""аварийный чек""",1
281,Nataliivanova,sberbank,None,1,0,оценка: 1 проверяется,Я являюсь поручителем по просроченному кредиту...,06.04.2017 22:01,Политика только Калужского отделения банка?,1
282,nastenka.eroha,sberbank,None,1,0,оценка: 1 проверяется,6 апреля 2017года обратилась в отделение №5221...,06.04.2017 21:47,Некомпетентность сотрудников!,1
287,sparkimus,sberbank,None,1,0,оценка: 3 проверяется,Добрый день!Меня зовут Ольга Александровна(Шве...,06.04.2017 21:42,Сколько можно ждать???!!!,3
288,Julia-2017,sberbank,None,1,0,оценка: 1 проверяется,Обратилась 6.04.2017 г. в отделение № 9038/051...,06.04.2017 21:05,"Некомпетентность сотрудников,обвинения в мошен...",1
289,zhozhooo,sberbank,None,1,0,оценка: 1 проверяется,"Получили выписку по карте и там сплошные ""Спис...",06.04.2017 20:27,Списание просроченных процентов за овердрафт,1
294,Detain,sberbank,None,1,0,оценка: 3 проверяется,В Сбербанке хорошего только сеть банкоматов и ...,06.04.2017 20:09,"Накипело... Три события, включая последнее.",3
295,М.Марков,sberbank,None,1,0,оценка: 1 проверяется,Однажды Сбербанк заблокировал мой расчетный сч...,06.04.2017 20:06,Прикрываясь 115-м ФЗ (беззастенчиво объявляя м...,1
301,EgorL,sberbank,None,1,5,оценка: 1 проверяется,**Сбербанк обнаглел!** Комиссию за оплату услу...,06.04.2017 17:54,Сбербанк обнаглел!!!,1


In [8]:
sentences_replies.head(10)

,sentence,reply_id,deleted_extra_symbols,lemmatized,vocab_only,vw
3814,Снимал деньги в банкомате 20.03.2017 в...,276,снимал деньги в банкомате в метро на пл,снимать деньги в банкомат в метро на пл,снимать деньги в банкомат в метро на пл,"{'банкомат': 2, 'снимать': 3, 'пл': 1, 'метро'..."
3815,"Александра Невского , банкомат денег не выдал ...",276,александра невского банкомат денег не выдал он...,имя невский банкомат деньги не выдавать он даж...,имя невский банкомат деньги не выдавать он даж...,"{'не': 3, 'банкомат': 2, 'невский': 1, 'деньги..."
3816,Сумма крупная.,276,сумма крупная,сумма крупный,сумма крупный,"{'сумма': 1, 'крупный': 1}"
3817,Пока шел пришла смс что деньги сняты.,276,пока шел пришла смс что деньги сняты,пока идти приходить смс что деньги снимать,пока идти приходить смс что деньги снимать,"{'снимать': 1, 'деньги': 1, 'пока': 1, 'приход..."
3818,Звонил по номеру 900 заявку приняли 1703200999...,276,звонил по номеру заявку приняли в работу срок ...,звонить по номер заявка принимать в работа сро...,звонить по номер заявка принимать в работа сро...,"{'постоянно': 1, 'время': 1, 'заявка': 1, 'пот..."
3819,Что бы я отказался от претензий?,276,что бы я отказался от претензий,что бы я отказываться от претензия,что бы я отказываться от претензия,"{'отказываться': 2, 'бы': 1, 'претензия': 1}"
3820,"Сегодня прислали смс ,что продлевают еще на 30...",276,сегодня прислали смс что продлевают еще на дней,сегодня присылать смс что продлевать еще на день,сегодня присылать смс что продлевать еще на день,"{'сегодня': 1, 'продлевать': 1, 'еще': 1, 'при..."
3821,Куда обращаться в полицию или в суд?,276,куда обращаться в полицию или в суд,куда обращаться в полиция или в суд,куда обращаться в полиция или в суд,"{'обращаться': 2, 'полиция': 1, 'куда': 1, 'су..."
3851,06.04.2017г.,280,г,г,г,NaN
3852,в 22:55 (дата указана на чеке) мною была внесе...,280,в дата указана на чеке мною была внесена сумма...,в дата указывать на чек я быть вносить сумма в...,в дата указывать на чек я быть вносить сумма в...,"{'дата': 1, 'вносить_сумма': 1, 'указывать': 3..."


In [9]:
stopWords = set(stopwords.words('russian'))

In [10]:
filtered_words = []
for sentence in sentences_replies.vocab_only:
    sentence_list = [word for word in sentence.split() if word not in stopWords]
    filtered_words.append(sentence_list)

sentences_replies['filtered'] = filtered_words
sentences_replies['filtered'] = sentences_replies['filtered'].apply(lambda x: ' '.join(x))
len(filtered_words)

835113

In [11]:
replies['filtered'] = sentences_replies.groupby('reply_id').aggregate({'filtered': lambda x: ' '.join(x)})

splitted_text = replies['filtered'].apply(lambda x: x.split())

In [14]:
dictionary = gensim.corpora.Dictionary(splitted_text)
print(len(dictionary))
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
print(len(dictionary))

12725
9856


In [16]:
bow_corpus = [dictionary.doc2bow(doc) for doc in splitted_text]


In [23]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=30, id2word=dictionary, passes=15, workers=2)


In [24]:
import pyLDAvis.gensim

vis_data = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)

In [25]:
pyLDAvis.display(vis_data)
